# MNIST 데이터셋에 차원축소 적용해보기

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1)
x, y = mnist["data"], mnist["target"]
x_train, x_test = x[: 60000], x[60000: ]
y_train, y_test = y[: 60000], y[60000: ]

In [13]:
from sklearn.ensemble import RandomForestClassifier
import time

rnd_clf1 = RandomForestClassifier()

In [15]:
t1 = time.time()
rnd_clf1.fit(x_train, y_train)
t2 = time.time()
print(t2 - t1)

35.20192527770996


In [16]:
x_train.shape, y_train.shape

((60000, 784), (60000,))

In [21]:
from sklearn.decomposition import PCA
import numpy as np

temp_pca = PCA()
temp_pca.fit(x_train)
cumsum = np.cumsum(temp_pca.explained_variance_ratio_)
target_d = np.argmax(cumsum >= 0.95) + 1
target_d

154

In [22]:
pca = PCA(n_components=target_d)
x_train_re = pca.fit_transform(x_train)

In [17]:
x_train_re.shape, y_train.shape

((60000, 154), (60000,))

In [18]:
rnd_clf2 = RandomForestClassifier()
t1 = time.time()
rnd_clf2.fit(x_train_re, y_train)
t2 = time.time()
print(t2 - t1)

72.09610748291016


?????????????  
왜 시간 늘어남????  
다행히 답안지에도 시간이 2배 이상 늘어났다고 함(휴).  
> **차원축소 알고리즘은 언제나 학습 시간을 줄여주지는 못함**  
데이터셋, 모델, 학습 알고리즘에 따라 달라짐.

실제로 랜덤포레스트가 아닌 소프트맥스 회귀에 적용하면 3배 이상 빨라진다고 함.

In [23]:
from sklearn.metrics import accuracy_score

pred1 = rnd_clf1.predict(x_test)

x_test_re = pca.transform(x_test)
pred2 = rnd_clf2.predict(x_test_re)

accuracy_score(y_test, pred1), accuracy_score(y_test, pred2)

(0.9696, 0.9489)

성능은 약간 떨어짐. 손실된 정보가 있으니 이는 당연한 결과임.